This file creates the parser and also created the chroma DB to be used in the main file.


In [1]:
import os
import chromadb
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.embeddings.openai import OpenAIEmbedding

from llama_index.core.vector_stores.types import VectorStoreQueryMode
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import Document
from llama_index.llms.openai import OpenAI

## import settings from llama_index
from llama_index.core import Settings
from chromadb.config import Settings as StChroma


In [2]:
# configure the llm and the embedding model
llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
)

Settings.llm = llm
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")


In [3]:
origin_path = "../data/md"

files_list = os.listdir(origin_path)


In [4]:
documents = []

for filename in files_list:
    with open(os.path.join(origin_path, filename), "r", encoding="utf-8") as file:
        readed = file.read()
        categories = ""
        categories_slug = ""
        preparation_time = ""
        cooking_time = ""
        difficulty = ""

        try:
            metadata, text = readed.split("---\n", 2)[1:]

            metadata_dict = {}
            metadata_lines = metadata.split("\n")
            for line in metadata_lines:
                if line.strip():  # Ensure the line is not empty
                    key, value = line.split(": ", 1)
                    metadata_dict[key] = value.strip()

            document = Document(text=text, metadata=metadata_dict)

            documents.append(document)
        except:
            print(f"not enough values in file {filename}. Something went wrong.")


In [5]:
print(documents)


[Document(id_='cb0a9f50-6f41-4270-923f-ab8afe92b010', embedding=None, metadata={'categories': 'Saludables', 'categories_slug': 'saludables', 'preparation_time': '30 mins', 'cooking_time': 'N/A', 'difficulty': 'Baja', 'ingredients': '4 tazas de harina, 1 cucharada de azúcar, 1 cucharada de levadura seca, suficiente de agua, a temperatura ambiente, suficiente de harina, para alimentar la masa, suficiente de agua, para alimentar la masa'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='\n# Como Hacer Masa Madre\n\nLa masa madre es un fermento que se prepara a base de harina y agua, para después dejarse fermentar de manera natural. Tradicionalmente, ha servido para fermentar el pan antes de que existiera la levadura comercial.Anteriormente, los panaderos solían guardar la masa madre, ya sea en estado líquido o como un trozo de masa, para la elaboración diaria del pan, incorporándole harina y los demás elementos, pero ahora es más común emplear levad

configure the llm


In [6]:
splitter = MarkdownNodeParser()

embed_model_name = "text-embedding-3-small"

embed_model = OpenAIEmbedding(
    model=embed_model_name,
)


In [7]:
chroma_client = chromadb.PersistentClient(
    path="./chroma_recipe", settings=StChroma(allow_reset=True)
)
# delete collection if it exists
if any(coll.name == "recipes" for coll in chroma_client.list_collections()):
    chroma_client.delete_collection("recipes")
chroma_collection = chroma_client.create_collection("recipes")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [8]:
# chroma_client = chromadb.EphemeralClient()
# # delete collection if it exists
# if any(coll.name == "test" for coll in chroma_client.list_collections()):
#     chroma_client.delete_collection("test")
# chroma_collection = chroma_client.create_collection("test")
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


In [9]:
pipeline = IngestionPipeline(
    transformations=[
        splitter,
        embed_model,
    ]
)


In [10]:
index = VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)


In [11]:
# run the pipeline to generate nodes
nodes = pipeline.run(documents=documents)


In [12]:
nodes[0]


TextNode(id_='f43024a2-0d65-4ff0-a354-e09be7ab1a06', embedding=[0.024201108142733574, -0.011722411960363388, 0.014391016215085983, 0.004119050223380327, -0.019025960937142372, -0.005172446835786104, -0.012943271547555923, 0.01702720858156681, -0.014520665630698204, -0.001511218841187656, 0.008762097917497158, -0.0796043649315834, 0.033427782356739044, -0.009183456189930439, 0.005899020470678806, 0.035134825855493546, 0.005501970648765564, -0.0009291497408412397, 0.0134186502546072, 0.01922043412923813, 0.03061872348189354, -0.03502678498625755, 0.021597329527139664, 0.028911681845784187, 0.003524826606735587, -0.01269477792084217, 0.008772902190685272, -0.012575933709740639, 0.009356321766972542, -0.01819404773414135, -0.032995618879795074, -0.04068811610341072, -0.036733824759721756, -0.016508612781763077, 0.020279232412576675, -0.07052894681692123, 0.014661118388175964, -0.015276949852705002, -0.017578216269612312, 0.001432889373973012, -0.05531682074069977, -0.010577181354165077, 0.

In [13]:
# Inserta nodos en el índice
index.insert_nodes(nodes)


In [14]:
engine = index.as_query_engine()


In [15]:
resp = engine.query("Recomiendame algo caliente para el frío")


In [16]:
print(resp)


Una excelente opción para el frío es el caldo Tlalpeño. Este platillo reconfortante, típico de la Ciudad de México, combina pollo, garbanzos, papas, zanahorias y un toque de chile chipotle adobado. Puedes acompañarlo con aguacate, queso Oaxaca y un poco de jugo de limón para realzar su sabor. Además, su preparación es sencilla y no toma mucho tiempo.


In [17]:
resp.response


'Una excelente opción para el frío es el caldo Tlalpeño. Este platillo reconfortante, típico de la Ciudad de México, combina pollo, garbanzos, papas, zanahorias y un toque de chile chipotle adobado. Puedes acompañarlo con aguacate, queso Oaxaca y un poco de jugo de limón para realzar su sabor. Además, su preparación es sencilla y no toma mucho tiempo.'